In [8]:
import os
import json
import openai
from rich import print
from dotenv import load_dotenv, find_dotenv

from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

load_dotenv(find_dotenv())

openai_api_key = os.environ.get("OPENAI_API_KEY")

In [117]:
system_message = SystemMessagePromptTemplate.from_template(
    """Your task is to act as a personal fitness trainer. 
    You will be provided with information about your CLIENT's current fitness level, their goals, and other preferences they have. 
    You have to prepare a detailed workout plan for the CLIENT, including everything essential for fitness.
    Ensure that you strictly adhere to the number of days and weeks the CLIENT is willing to do their workout."""  # noqa: E501
)

In [118]:
with open('./questions.json') as f:
    questions_list = json.load(f)['questions']

questions_list

[{'id': 1,
  'section': 'Personal characteristics',
  'text': 'What is your age range?',
  'question_type': 'single_select',
  'options': ['Under 18',
   '18-24',
   '25-34',
   '35-44',
   '45-54',
   '55-64',
   '65 and above']},
 {'id': 2,
  'section': 'Personal characteristics',
  'text': 'What is your gender?',
  'question_type': 'single_select',
  'options': ['Male', 'Female', 'Non-binary', 'Prefer not to say']},
 {'id': 3,
  'section': 'Personal characteristics',
  'text': 'What is your primary goal for strength training?',
  'question_type': 'single_select',
  'options': ['Increase muscle mass',
   'Improve overall strength',
   'Improve athletic performance',
   'Lose body fat and tone muscles',
   'Rehabilitation or injury prevention']},
 {'id': 4,
  'section': 'Personal characteristics',
  'text': 'How would you describe your current fitness level?',
  'question_type': 'single_select',
  'options': ['Beginner (little or no experience)',
   'Intermediate (some experience)',
 

In [11]:
question_message = HumanMessagePromptTemplate.from_template(
    f'The list of questions that have been asked to the CLIENT (separated by a ;) are: \n{";".join(i["text"] for i in questions_list)}'
)



In [12]:
print(question_message)

prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='The list of questions
that have been asked to the CLIENT (separated by a ;) are: \nWhat is your age range?;What is your gender?;What is 
your primary goal for strength training?;How would you describe your current fitness level?;What is your current 
height (in cms)?;What is your current weight (in kgs)?;Have you ever followed a structured strength training 
program before?;Do you have any previous injuries or medical conditions that may affect your training?;How many 
days per week can you dedicate to strength training?;How many weeks do you plan to workout?;Do you have access to a
gym or will you be working out at home?;Which of these muscle groups do you consider your weakest or want to 
improve the most?;How would you rate your experience with strength training exercises?;What type of strength 
training program do you prefer?;Do you prefer to focus on bodyweight exercises, free weights, or a mix of both?;How
important is increasing your cardiovascular fitness to you?', template_format='f-string', validate_template=True) 
additional_kwargs={}

In [13]:
answer_message = HumanMessagePromptTemplate.from_template(
    'The answers provided by the CLIENT (separated by a ;) are: \nIncrease muscle mass;Beginner (little or no experience);5\'10";78 kgs;Yes;No;4 days;6 weeks;Gym access;Upper body;Core'
)

In [14]:
print(answer_message.prompt.template)

The answers provided by the CLIENT (separated by a ;) are: 
Increase muscle mass;Beginner (little or no experience);5'10";78 kgs;Yes;No;4 days;6 weeks;Gym access;Upper 
body;Core

In [67]:
from typing import List
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class Duration(BaseModel):
    value: int = Field(description="numerical value of a duration")
    unit: str = Field(description="the unit of time of a duration")

class Exercise(BaseModel):
    name: str = Field(description="name of the exercise")
    exercise_type: str = Field(default=None, description="type of exercise (e.g., warmup, cardio, stretching, strength)")
    sets: int = Field(default=None, description="number of sets to perform")
    reps: int = Field(default=None, description="number of repetitions per set")
    duration: Duration = Field(default=None, description="duration of the exercise, if applicable")

class Day(BaseModel):
    day_number: int = Field(description="the day number within the workout plan")
    focus: str = Field(description="primary focus area of the day")
    exercises: List[Exercise] = Field(default=None, description="list of exercises to be performed")

class Week(BaseModel):
    week_range: str = Field(description="week numbers the workout plan has to be followed (e.g., Week 1-4, Week 5-7)")
    days: List[Day] = Field("list of days in the workout plan for specific week")
    
class Note(BaseModel):
    content: str = Field(description="the content of the note")

class WorkoutPlan(BaseModel):
    weeks: List[Week] = Field("list of weeks in the workout plan")
    notes: List[Note] = Field("list of notes for the workout plan")

In [9]:
parser = PydanticOutputParser(pydantic_object=WorkoutPlan)

In [16]:
format_message = HumanMessagePromptTemplate.from_template(
    template="""With the given details, proceed to create a detailed weekly plan that can be followed for 2 months for the CLIENT, making sure to incorporate their inputs properly.
    Ensure that you include cardio and rest time inside the plan as part of the `exercise` object. Try to keep the notes section as generic as possible as most CLIENTS will not read it.
    {format_instructions}""",
)


In [11]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message, question_message, answer_message, format_message]
)

messages = chat_prompt.format_prompt(format_instructions=parser.get_format_instructions()).to_messages()

print(messages)

[
    SystemMessage(content="Your task is to act as a personal fitness trainer. You will be provided with information
about your CLIENT's current fitness level, their goals, and other preferences they have. You have to prepare a 
detailed workout plan for the CLIENT, including everything essential for fitness.", additional_kwargs={}),
    HumanMessage(content='The list of questions that have been asked to the CLIENT (separated by a ;) are: \nWhat 
is your primary goal for strength training?;How would you describe your current fitness level?;What is your current
height?;What is your current weight?;Have you ever followed a structured strength training program before?;Do you 
have any previous injuries or medical conditions that may affect your training?;How many days per week can you 
dedicate to strength training?;How many weeks do you plan to workout?;Do you have access to a gym or will you be 
working out at home?;Which of these muscle groups do you consider your strongest?;Which of these muscle groups do 
you consider your weakest or want to improve the most?', additional_kwargs={}, example=False),
    HumanMessage(content='The answers provided by the CLIENT (separated by a ;) are: \nIncrease muscle 
mass;Beginner (little or no experience);5\'10";78 kgs;Yes;No;4 days;6 weeks;Gym access;Upper body;Core', 
additional_kwargs={}, example=False),
    HumanMessage(content='With the given details, proceed to create a detailed weekly plan that can be followed for
2 months for the CLIENT, making sure to incorporate their inputs properly.\n    Ensure that you include cardio and 
rest time inside the plan as part of the `exercise` object. Try to keep the notes section as generic as possible as
most CLIENTS will not read it.\n    The output should be formatted as a JSON instance that conforms to the JSON 
schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of 
strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]}
is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not 
well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"weeks": {"title": "Weeks", "default": "list of
weeks in the workout plan", "type": "array", "items": {"$ref": "#/definitions/Week"}}, "notes": {"title": "Notes", 
"default": "list of notes for the workout plan", "type": "array", "items": {"$ref": "#/definitions/Note"}}}, 
"definitions": {"Duration": {"title": "Duration", "type": "object", "properties": {"value": {"title": "Value", 
"description": "numerical value of a duration", "type": "integer"}, "unit": {"title": "Unit", "description": "the 
unit of time of a duration", "type": "string"}}, "required": ["value", "unit"]}, "Exercise": {"title": "Exercise", 
"type": "object", "properties": {"name": {"title": "Name", "description": "name of the exercise", "type": 
"string"}, "exercise_type": {"title": "Exercise Type", "description": "type of exercise (e.g., warmup, cardio, 
stretching, strength)", "type": "string"}, "sets": {"title": "Sets", "description": "number of sets to perform", 
"type": "integer"}, "reps": {"title": "Reps", "description": "number of repetitions per set", "type": "integer"}, 
"duration": {"title": "Duration", "description": "duration of the exercise, if applicable", "allOf": [{"$ref": 
"#/definitions/Duration"}]}}, "required": ["name"]}, "Day": {"title": "Day", "type": "object", "properties": 
{"day_number": {"title": "Day Number", "description": "the day number within the workout plan", "type": "integer"},
"focus": {"title": "Focus", "description": "primary focus area of the day", "type": "string"}, "exercises": 
{"title": "Exercises", "description": "list of exercises to be performed", "type": "array", "items": {"$ref": 
"#/definitions/Exercise"}}}, "required": ["day_number", "focus"]}, "Week": {"title": "Week", "type": "object", 
"properties": {"week_range": {"title

In [107]:
chat = ChatOpenAI(model_name="gpt-4", temperature=0.4, openai_api_key=openai_api_key, verbose=True)


In [12]:
chat_turbo = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.4, openai_api_key=openai_api_key, verbose=True)


In [13]:
response = chat(messages)

In [17]:
print(response.content)

{
  "weeks": [
    {
      "week_range": "Week 1-4",
      "days": [
        {
          "day_number": 1,
          "focus": "Upper Body",
          "exercises": [
            {
              "name": "Bench Press",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 10
            },
            {
              "name": "Pull-ups",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 8
            },
            {
              "name": "Dumbbell Shoulder Press",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 10
            },
            {
              "name": "Tricep Dips",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 12
            },
            {
              "name": "Bicep Curls",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 12
            }
          ]
        },
        {
          "day_number": 2,
          "focus": "Cardio",
          "exercises": [
            {
              "name": "Treadmill",
              "exercise_type": "Cardio",
              "duration": {
                "value": 30,
                "unit": "minutes"
              }
            }
          ]
        },
        {
          "day_number": 3,
          "focus": "Rest",
          "exercises": []
        },
        {
          "day_number": 4,
          "focus": "Core",
          "exercises": [
            {
              "name": "Plank",
              "exercise_type": "Strength",
              "sets": 3,
              "duration": {
                "value": 60,
                "unit": "seconds"
              }
            },
            {
              "name": "Russian Twists",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 15
            },
            {
              "name": "Leg Raises",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 12
            },
            {
              "name": "Bicycle Crunches",
              "exercise_type": "Strength",
              "sets": 3,
              "reps": 15
            }
          ]
        }
      ]
    },
    {
      "week_range": "Week 5-8",
      "days": [
        {
          "day_number": 1,
          "focus": "Upper Body",
          "exercises": [
            {
              "name": "Bench Press",
              "exercise_type": "Strength",
              "sets": 4,
              "reps": 10
            },
            {
              "name": "Pull-ups",
              "exercise_type": "Strength",
              "sets": 4,
              "reps": 8
            },
            {
              "name": "Dumbbell Shoulder Press",
              "exercise_type": "Strength",
              "sets": 4,
              "reps": 10
            },
            {
              "name": "Tricep Dips",
              "exercise_type": "Strength",
              "sets": 4,
              "reps": 12
            },
            {
              "name": "Bicep Curls",
              "exercise_type": "Strength",
              "sets": 4,
              "reps": 12
            }
          ]
        },
        {
          "day_number": 2,
          "focus": "Cardio",
          "exercises": [
            {
              "name": "Treadmill",
              "exercise_type": "Cardio",
              "duration": {
                "value": 40,
                "unit": "minutes"
              }
            }
          ]
        },
        {
          "day_number": 3,
          "focus": "Rest",
          "exercises": []
        },
        {
          "day_number": 4,
          "focus": "Core",
          "exercises": [
            {
              "name": "Plank",
              "exercise_type": "Strength",
              "sets": 4,
              "duration": {
                "value": 60,
                "unit": "seconds"
              }
            },
            {
   

In [18]:
plan  = parser.parse(response.content)
print(plan)

weeks=[Week(week_range='Week 1-4', days=[Day(day_number=1, focus='Upper Body', exercises=[Exercise(name='Bench 
Press', exercise_type='Strength', sets=3, reps=10, duration=None), Exercise(name='Pull-ups', 
exercise_type='Strength', sets=3, reps=8, duration=None), Exercise(name='Dumbbell Shoulder Press', 
exercise_type='Strength', sets=3, reps=10, duration=None), Exercise(name='Tricep Dips', exercise_type='Strength', 
sets=3, reps=12, duration=None), Exercise(name='Bicep Curls', exercise_type='Strength', sets=3, reps=12, 
duration=None)]), Day(day_number=2, focus='Cardio', exercises=[Exercise(name='Treadmill', exercise_type='Cardio', 
sets=None, reps=None, duration=Duration(value=30, unit='minutes'))]), Day(day_number=3, focus='Rest', 
exercises=[]), Day(day_number=4, focus='Core', exercises=[Exercise(name='Plank', exercise_type='Strength', sets=3, 
reps=None, duration=Duration(value=60, unit='seconds')), Exercise(name='Russian Twists', exercise_type='Strength', 
sets=3, reps=15, duration=None), Exercise(name='Leg Raises', exercise_type='Strength', sets=3, reps=12, 
duration=None), Exercise(name='Bicycle Crunches', exercise_type='Strength', sets=3, reps=15, duration=None)])]), 
Week(week_range='Week 5-8', days=[Day(day_number=1, focus='Upper Body', exercises=[Exercise(name='Bench Press', 
exercise_type='Strength', sets=4, reps=10, duration=None), Exercise(name='Pull-ups', exercise_type='Strength', 
sets=4, reps=8, duration=None), Exercise(name='Dumbbell Shoulder Press', exercise_type='Strength', sets=4, reps=10,
duration=None), Exercise(name='Tricep Dips', exercise_type='Strength', sets=4, reps=12, duration=None), 
Exercise(name='Bicep Curls', exercise_type='Strength', sets=4, reps=12, duration=None)]), Day(day_number=2, 
focus='Cardio', exercises=[Exercise(name='Treadmill', exercise_type='Cardio', sets=None, reps=None, 
duration=Duration(value=40, unit='minutes'))]), Day(day_number=3, focus='Rest', exercises=[]), Day(day_number=4, 
focus='Core', exercises=[Exercise(name='Plank', exercise_type='Strength', sets=4, reps=None, 
duration=Duration(value=60, unit='seconds')), Exercise(name='Russian Twists', exercise_type='Strength', sets=4, 
reps=15, duration=None), Exercise(name='Leg Raises', exercise_type='Strength', sets=4, reps=12, duration=None), 
Exercise(name='Bicycle Crunches', exercise_type='Strength', sets=4, reps=15, duration=None)])])] 
notes=[Note(content='Remember to warm up for 5-10 minutes before each workout session.'), Note(content='Cool down 
and stretch for 5-10 minutes after each workout session.'), Note(content='Progress at your own pace and listen to 
your body.'), Note(content='Maintain proper form and technique during exercises to avoid injuries.'), 
Note(content='Stay consistent with your workouts and nutrition for optimal results.')]

In [2]:
def print_plan(plan):
    for p in plan.weeks:
        print(f"Week: {p.week_range}")
        for day in p.days:
            print(f"Day: {day.day_number}\nFocus: {day.focus}")
            print([f"{exercise.name} -> {exercise.sets} x {exercise.reps} x {exercise.duration}" for exercise in day.exercises])

    print(plan.notes)

print_plan(plan)

NameError: name 'plan' is not defined

In [30]:
weeks = WorkoutPlan(
    weeks=[
        Week(
            week_range="Week 1-2",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Upper Body (Chest, Shoulders, Triceps)",
                    exercises=[
                        Exercise(
                            name="Push-ups",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Dumbbell Shoulder Press",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Tricep Dips",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bench Dips",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=20, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=20, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
        Week(
            week_range="Week 3-4",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=20,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Lower Body (Legs, Glutes)",
                    exercises=[
                        Exercise(
                            name="Squats",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Lunges",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Leg Press",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Calf Raises",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=25, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=25, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
        Week(
            week_range="Week 5-6",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=25,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Upper Body (Back, Biceps)",
                    exercises=[
                        Exercise(
                            name="Pull-ups",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                        Exercise(
                            name="Dumbbell Rows",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicep Curls",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Hammer Curls",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=30, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=30, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
        Week(
            week_range="Week 7-8",
            days=[
                Day(
                    day_number=1,
                    focus="Core (abs, lower back)",
                    exercises=[
                        Exercise(
                            name="Plank",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=Duration(value=30, unit="seconds"),
                        ),
                        Exercise(
                            name="Crunches",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=None,
                        ),
                        Exercise(
                            name="Russian Twist",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=None,
                        ),
                        Exercise(
                            name="Bicycle Crunch",
                            exercise_type="Strength",
                            sets=3,
                            reps=30,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=2,
                    focus="Full Body",
                    exercises=[
                        Exercise(
                            name="Deadlifts",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                        Exercise(
                            name="Squats",
                            exercise_type="Strength",
                            sets=3,
                            reps=15,
                            duration=None,
                        ),
                        Exercise(
                            name="Bench Press",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                        Exercise(
                            name="Pull-ups",
                            exercise_type="Strength",
                            sets=3,
                            reps=10,
                            duration=None,
                        ),
                    ],
                ),
                Day(
                    day_number=3,
                    focus="Cardio",
                    exercises=[
                        Exercise(
                            name="Running",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=35, unit="minutes"),
                        ),
                        Exercise(
                            name="Jumping Jacks",
                            exercise_type="Cardio",
                            sets=None,
                            reps=None,
                            duration=Duration(value=35, unit="minutes"),
                        ),
                    ],
                ),
            ],
        ),
    ],
    notes=[
        Note(
            content="Remember to stretch before and after each workout to prevent injury."
        ),
        Note(
            content="Make sure to stay hydrated throughout the day and during workouts."
        ),
        Note(
            content="If you experience any pain or discomfort during exercises, stop immediately and consult a doctor."
        ),
    ],
)

In [99]:
import pandas as pd

print(weeks.notes)

[
    Note(content='Remember to stretch before and after each workout to prevent injury.'),
    Note(content='Make sure to stay hydrated throughout the day and during workouts.'),
    Note(content='If you experience any pain or discomfort during exercises, stop immediately and consult a 
doctor.')
]

In [101]:
def convert_to_dataframe(plan):
    weeks, notes = plan.weeks, plan.notes

    workout_data = []
    for week in weeks:
        week_range, days = week.week_range, week.days
        for day in days:
            for exercise in day.exercises:
                row = {
                    "Week Range": week_range,
                    "Day Number": day.day_number,
                    "Focus": day.focus,
                    "Exercise Name": exercise.name,
                    "Exercise Type": exercise.exercise_type,
                    "Sets": exercise.sets,
                    "Reps": exercise.reps,
                    "Duration": f"{exercise.duration.value} {exercise.duration.unit}"
                    if exercise.duration
                    else None,
                }
                workout_data.append(row)

    note_data = []
    for note in notes:
        row = {"note": note.content}
        note_data.append(row)

    return workout_data, note_data

In [104]:
w,n = convert_to_dataframe(weeks)
pd.DataFrame(n)

,note
0,Remember to stretch before and after each work...
1,Make sure to stay hydrated throughout the day ...
2,If you experience any pain or discomfort durin...


In [119]:
messages = [
    SystemMessage(
        content="Your task is to act as a personal fitness trainer. \n    You will be provided with information about your CLIENT's current fitness level, their goals, and other preferences they have. \n    You have to prepare a detailed workout plan for the CLIENT, including everything essential for fitness.\n    Ensure that you strictly adhere to the number of days and weeks the CLIENT is willing to do their workout.",
        additional_kwargs={},
    ),
    HumanMessage(
        content="The list of questions that have been asked to the CLIENT (separated by a ;) are: \nWhat is your age range?;What is your gender?;What is your primary goal for strength training?;How would you describe your current fitness level?;What is your current height (in cms)?;What is your current weight (in kgs)?;Have you ever followed a structured strength training program before?;Do you have any previous injuries or medical conditions that may affect your training?;How many days per week can you dedicate to strength training?;How many weeks do you plan to workout?;Do you have access to a gym or will you be working out at home?;Which of these muscle groups do you consider your weakest or want to improve the most?;How would you rate your experience with strength training exercises?;What type of strength training program do you prefer?;Do you prefer to focus on bodyweight exercises, free weights, or a mix of both?;How important is increasing your cardiovascular fitness to you?",
        additional_kwargs={},
        example=False,
    ),
    HumanMessage(
        content="The answers provided by the CLIENT (separated by a ;) are: \n18-24;Male;Lose body fat and tone muscles;Beginner (little or no experience);180.0;78.9;No;No;Core (abs, lower back);Beginner (little to no experience);5 days;4 weeks;Gym access;Body part specific split (e.g. chest day, back day);A mix of both;Not important",
        additional_kwargs={},
        example=False,
    ),
    HumanMessage(
        content='With the given details, proceed to create a detailed weekly plan that can be followed for 2 months for the CLIENT, making sure to incorporate their inputs properly.\n    Ensure that you include cardio and rest time inside the plan as part of the `exercise` object. Try to keep the notes section as generic as possible as most CLIENTS will not read it.\n    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"weeks": {"title": "Weeks", "default": "list of weeks in the workout plan", "type": "array", "items": {"$ref": "#/definitions/Week"}}, "notes": {"title": "Notes", "default": "list of notes for the workout plan", "type": "array", "items": {"$ref": "#/definitions/Note"}}}, "definitions": {"Duration": {"title": "Duration", "type": "object", "properties": {"value": {"title": "Value", "description": "numerical value of a duration", "type": "integer"}, "unit": {"title": "Unit", "description": "the unit of time of a duration", "type": "string"}}, "required": ["value", "unit"]}, "Exercise": {"title": "Exercise", "type": "object", "properties": {"name": {"title": "Name", "description": "name of the exercise", "type": "string"}, "exercise_type": {"title": "Exercise Type", "description": "type of exercise (e.g., warmup, cardio, stretching, strength)", "type": "string"}, "sets": {"title": "Sets", "description": "number of sets to perform", "type": "integer"}, "reps": {"title": "Reps", "description": "number of repetitions per set", "type": "integer"}, "duration": {"title": "Duration", "description": "duration of the exercise, if applicable", "allOf": [{"$ref": "#/definitions/Duration"}]}}, "required": ["name"]}, "Day": {"title": "Day", "type": "object", "properties": {"day_number": {"title": "Day Number", "description": "the day number within the workout plan", "type": "integer"}, "focus": {"title": "Focus", "description": "primary focus area of the day", "type": "string"}, "exercises": {"title": "Exercises", "description": "list of exercises to be performed", "type": "array", "items": {"$ref": "#/definitions/Exercise"}}}, "required": ["day_number", "focus"]}, "Week": {"title": "Week", "type": "object", "properties": {"week_range": {"title": "Week Range", "description": "week numbers the workout plan has to be followed (e.g., Week X-Y)", "type": "string"}, "days": {"title": "Days", "default": "list of days in the workout plan for specific week", "type": "array", "items": {"$ref": "#/definitions/Day"}}}, "required": ["week_range"]}, "Note": {"title": "Note", "type": "object", "properties": {"content": {"title": "Content", "description": "the content of the note", "type": "string"}}, "required": ["content"]}}}\n```',
        additional_kwargs={},
        example=False,
    ),
]

In [120]:
messages

[SystemMessage(content="Your task is to act as a personal fitness trainer. \n    You will be provided with information about your CLIENT's current fitness level, their goals, and other preferences they have. \n    You have to prepare a detailed workout plan for the CLIENT, including everything essential for fitness.\n    Ensure that you strictly adhere to the number of days and weeks the CLIENT is willing to do their workout.", additional_kwargs={}),
 HumanMessage(content='The list of questions that have been asked to the CLIENT (separated by a ;) are: \nWhat is your age range?;What is your gender?;What is your primary goal for strength training?;How would you describe your current fitness level?;What is your current height (in cms)?;What is your current weight (in kgs)?;Have you ever followed a structured strength training program before?;Do you have any previous injuries or medical conditions that may affect your training?;How many days per week can you dedicate to strength training?

In [121]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.4,
    openai_api_key=openai_api_key,
    verbose=True,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [122]:
resp = chat(messages)

{
  "weeks": [
    {
      "week_range": "Week 1-2",
      "days": [
        {
          "day_number": 1,
          "focus": "Chest and Triceps",
          "exercises": [
            {
              "name": "Warmup - Jumping Jacks",
              "exercise_type": "warmup",
              "duration": {
                "value": 5,
                "unit": "minutes"
              }
            },
            {
              "name": "Bench Press",
              "exercise_type": "strength",
              "sets": 3,
              "reps": 12
            },
            {
              "name": "Pushups",
              "exercise_type": "strength",
              "sets": 3,
              "reps": 12
            },
            {
              "name": "Tricep Dips",
              "exercise_type": "strength",
              "sets": 3,
              "reps": 12
            },
            {
              "name": "Cooldown - Stretching",
              "exercise_type": "cooldown",
              "duration": {


ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))